<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-2-public/blob/adding_C2/C2/W2/assignment_optional/C2_W2_Assignment_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KEMANAKITA MACHINE LEARNING MODEL

## Setup

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import os

import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tqdm import tqdm

print("\u2022 Using TensorFlow Version:", tf.__version__)
print("\u2022 Using TensorFlow Hub Version: ", hub.__version__)
print(
    "\u2022 GPU Device Found."
    if tf.test.is_gpu_available()
    else "\u2022 GPU Device Not Found. Running on CPU"
)

• Using TensorFlow Version: 2.8.0
• Using TensorFlow Hub Version:  0.12.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
• GPU Device Found.


## Select the Hub/TF2 Module to Use

Hub modules for TF 1.x won't work here, please use one of the selections provided.

In [ ]:
module_selection = (
    "mobilenet_v2",
    224,
    1280,
)  # @param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(
    handle_base
)
IMAGE_SIZE = (pixels, pixels)
print(
    "Using {} with input size {} and output dimension {}".format(
        MODULE_HANDLE, IMAGE_SIZE, FV_SIZE
    )
)

Using https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 with input size (224, 224) and output dimension 1280


## Data Processing

In [ ]:
path = os.path.join(os.getcwd(), "gdrive", "MyDrive")  # comment this for local use
datapath = os.path.join(path, "Dataset")  # change path to os.getcwd()
train_path = os.path.join(datapath, "train")
test_path = os.path.join(datapath, "test")

In [ ]:
BATCH_SIZE = 10
IMG_SIZE = (224, 224)

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_path, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE
)
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_path, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE
)

Found 452 files belonging to 3 classes.
Found 33 files belonging to 3 classes.


In [ ]:
rescale = tf.keras.layers.Rescaling(1.0 / 127.5, offset=-1)

In [ ]:
BATCH_SIZE = 32  # @param {type:"integer"}

In [ ]:
num_examples = 452
num_classes = 3

In [ ]:
train_batches = train_dataset.shuffle(num_examples // 4).batch(BATCH_SIZE).prefetch(1)
validation_batches = test_dataset.batch(BATCH_SIZE).prefetch(1)
# test_batches = test_examples.batch(1).map(format_image)

## Defining the Model

All it takes is to put a linear classifier on top of the `feature_extractor_layer` with the Hub module.

For speed, we start out with a non-trainable `feature_extractor_layer`, but you can also enable fine-tuning for greater accuracy.

In [ ]:
do_fine_tuning = False  # @param {type:"boolean"}

In [ ]:
feature_extractor = hub.KerasLayer(
    MODULE_HANDLE,
    input_shape=IMAGE_SIZE + (3,),
    output_shape=[FV_SIZE],
    trainable=do_fine_tuning,
)

In [ ]:
print("Building model with", MODULE_HANDLE)

model = tf.keras.Sequential(
    [feature_extractor, tf.keras.layers.Dense(3, activation="softmax")]
)

model.summary()

Building model with https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 3)                 3843      
                                                                 
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
# @title (Optional) Unfreeze some layers
NUM_LAYERS = 17  # @param {type:"slider", min:1, max:50, step:1}

if do_fine_tuning:
    feature_extractor.trainable = True

    for layer in model.layers[-NUM_LAYERS:]:
        layer.trainable = True

else:
    feature_extractor.trainable = False

## Training the Model

In [ ]:
if do_fine_tuning:
    model.compile(
        optimizer=tf.keras.optimizers.SGD(lr=0.002, momentum=0.9),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )
else:
    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )

In [ ]:
EPOCHS = 20

hist = model.fit(train_dataset, epochs=EPOCHS, validation_data=test_dataset)

Epoch 1/20
46/46 [==============================] - 21s 156ms/step - loss: 1.2114 - accuracy: 0.4712 - val_loss: 0.6732 - val_accuracy: 0.7879
Epoch 2/20
46/46 [==============================] - 3s 51ms/step - loss: 0.6570 - accuracy: 0.7544 - val_loss: 0.5142 - val_accuracy: 0.8788
Epoch 3/20
46/46 [==============================] - 3s 50ms/step - loss: 0.5333 - accuracy: 0.8208 - val_loss: 0.4284 - val_accuracy: 0.8788
Epoch 4/20
46/46 [==============================] - 3s 50ms/step - loss: 0.4484 - accuracy: 0.8650 - val_loss: 0.3583 - val_accuracy: 0.9394
Epoch 5/20
46/46 [==============================] - 3s 52ms/step - loss: 0.4096 - accuracy: 0.8584 - val_loss: 0.3273 - val_accuracy: 0.9091
Epoch 6/20
46/46 [==============================] - 3s 51ms/step - loss: 0.3690 - accuracy: 0.8783 - val_loss: 0.2923 - val_accuracy: 0.9091
Epoch 7/20
46/46 [==============================] - 3s 52ms/step - loss: 0.3215 - accuracy: 0.9226 - val_loss: 0.2771 - val_accuracy: 0.9394
Epoch 8/20


## Export the Model

In [ ]:
RPS_SAVED_MODEL = "rps_saved_model"

Export the SavedModel

In [ ]:
tf.saved_model.save(model, RPS_SAVED_MODEL)

INFO:tensorflow:Assets written to: rps_saved_model/assets


INFO:tensorflow:Assets written to: rps_saved_model/assets


In [ ]:
%%bash -s $RPS_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['keras_layer_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_keras_layer_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 3)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [ ]:
loaded = tf.saved_model.load(RPS_SAVED_MODEL)

In [ ]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

['serving_default']
((), {'keras_layer_input': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_layer_input')})
{'dense': TensorSpec(shape=(None, 3), dtype=tf.float32, name='dense')}


## Convert Using TFLite's Converter

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(RPS_SAVED_MODEL)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

In [ ]:
tflite_model_file = "converted_model.tflite"

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

In [ ]:
import shutil

shutil.make_archive("rps_saved_model", "zip", "rps_saved_model")

'/content/rps_saved_model.zip'